## # Notes 01

## # 1.0 - General

In [ ]:
import math
import keras
import tensorflow
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split

from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator 
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Input, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.models import Sequential, model_from_json, Model
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

from keras.datasets import mnist, cifar10

from keras.preprocessing import image

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
def linearActivation(values):
    return values

def stepFunction(values):
    if values > 1:
        return 1
    return 0

def sigmoidActivation(values):
    return 1 / (1 + np.exp(-1 * values))

def tanhActivation(values):
    return (np.exp(values) - np.exp(-values)) / (np.exp(values) + np.exp(-1 * values))

def reLU(values):
    if values >= 0:
        return values
    return 0

def leakyReLU(values, leaky=0.01):
    if values >= 0:
        return values
    return leaky * values

def softmax(x):
    ex = np.exp(x)
    return ex/ex.sum()

In [ ]:
def MAE(erros):
    return (1/len(erros)) * math.sqrt(sum([abs(e) for e in erros]))

def MSE(erros):
    return (1/len(erros)) * sum([e**2 for e in erros])

def RMSE(erros):
    return math.sqrt((1/len(erros)) * sum([e**2 for e in erros]))

def accuracy(tp, tn, total):
    return (tp + tn)/total

In [ ]:
_0 = (5*2 + 2*5 + 1*1)/10
_1 = np.array([5.0, 2.0, 1.3])
_2 = [1 - 0.3, 0 - 0.2, 1 - 0.89, 0 - 0.32]

In [ ]:
(
    f'Linear: {linearActivation(_0)}',
    f'Step: {stepFunction(_0)}',
    f'Sigmoid: {sigmoidActivation(_0)}',
    f'TanH: {tanhActivation(_0)}',
    f'ReLU: {reLU(_0)}',
    f'Leaky ReLU:{leakyReLU(_0)}',
    f'Softmax: {softmax(_1)}'
)

In [ ]:
(
    f'MAE: {MAE(_2)}',
    f'MSE: {MSE(_2)}',
    f'RMSE: {RMSE(_2)}',
    f'Acc: {accuracy(1, 2, 4)}'
)

In [ ]:
def nans(df):
    return df[df.isnull().any(axis=1)]

## # 2.0 - Breast Cancer

In [ ]:
df = pd.read_csv('datasets/breast_input.csv')

In [ ]:
df.head()

In [ ]:
classes = pd.read_csv('datasets/breast_classes.csv')

In [ ]:
classes.head()

In [ ]:
trainX, testX, trainY, testY = train_test_split(df, classes,
                                                test_size=0.25,
                                                random_state=0)

In [ ]:
f'trainX: {trainX.shape}, trainY:{trainY.shape}, \
testX: {testX.shape}, testY: {testY.shape}'

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(units=30,
                activation='relu',
                kernel_initializer='random_uniform',
                input_dim=30,
                name='layer_1'))

In [ ]:
model.add(Dense(units=30,
                activation='relu',
                kernel_initializer='random_uniform',
                name='layer_2'))

In [ ]:
model.add(Dense(units=10,
                activation='relu',
                kernel_initializer='random_uniform',
                name='layer_3'))

In [ ]:
model.add(Dense(units=10,
                activation='relu',
                kernel_initializer='random_uniform',
                name='layer_4'))

In [ ]:
model.add(Dense(units=1, activation='sigmoid', name='output'))

In [ ]:
my_optimizer = keras.optimizers.Adam(lr=0.01, decay=0.0001, clipvalue=0.5)

In [ ]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x=trainX, y=trainY, epochs=100, verbose=1, shuffle=True)

In [ ]:
pred = model.predict(x=testX)

In [ ]:
pred = (pred > 0.5)

In [ ]:
results = model.evaluate(x=testX, y=testY)
results

In [ ]:
acc = accuracy_score(testY, pred)
acc

In [ ]:
matrix = confusion_matrix(testY, pred)
matrix

In [ ]:
recall = recall_score(testY, pred)
recall

In [ ]:
precision = precision_score(testY, pred)
precision

In [ ]:
w0, b0 = model.layers[0].get_weights()[0], model.layers[0].get_weights()[1]
w1, b1 = model.layers[1].get_weights()[0], model.layers[1].get_weights()[1]
w1, b1 = model.layers[1].get_weights()[0], model.layers[1].get_weights()[1]
w2, b2 = model.layers[2].get_weights()[0], model.layers[2].get_weights()[1]
w3, b3 = model.layers[3].get_weights()[0], model.layers[3].get_weights()[1]
w4, b4 = model.layers[4].get_weights()[0], model.layers[4].get_weights()[1]

In [ ]:
w0.shape, b0.shape

In [ ]:
w1.shape, b1.shape

In [ ]:
w2.shape, b2.shape

In [ ]:
w3.shape, b3.shape

In [ ]:
w4.shape, b4.shape

## # 1.1 - Predict new data

In [ ]:
new_data = np.array(
    [
        [
            15.8, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178, 0.20,
            0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.014, 0.03,
            0.007, 23.15, 16.63, 178.5, 2018, 0.14, 0.185, 0.84, 158, 0.363
        ]
    ]
)

new_data.shape

In [ ]:
# since sigmoide is probabilistic, they'll have the same values
f'pred: {model.predict(x=new_data)[0][0]}, pred_proba: {model.predict_proba(x=new_data)[0][0]}'

## # 1.2 - Export the network

In [ ]:
with open('model_breast.json', mode='w') as f:
    f.write(model.to_json())

In [ ]:
model.save_weights('model_breast.h5')

In [ ]:
!ls -laH

## # 1.3 - Import the network

In [ ]:
json_model_str = None
with open('model_breast.json', mode='r') as f:
    json_model_str = f.read()    

In [ ]:
json_model_str

In [ ]:
json_model = model_from_json(json_string=json_model_str)
json_model

In [ ]:
json_model.load_weights('model_breast.h5')

In [ ]:
json_model.predict(x=new_data)[0][0], json_model.predict_proba(x=new_data)[0][0]

In [ ]:
json_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [ ]:
json_model.evaluate(x=df, y=classes)

## # 2.1 - K-fold

In [ ]:
def createNetowrk():
    clf = Sequential()
    clf.add(Dense(units=30, activation='relu',kernel_initializer='random_uniform', input_dim=30))
    clf.add(Dropout(rate=0.2, seed=0))
    clf.add(Dense(units=30, activation='relu', kernel_initializer='random_uniform'))
    clf.add(Dropout(rate=0.2, seed=0))
    clf.add(Dense(units=30, activation='relu', kernel_initializer='random_uniform'))
    clf.add(Dropout(rate=0.2, seed=0))
    clf.add(Dense(units=30, activation='relu', kernel_initializer='random_uniform'))
    clf.add(Dense(units=1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(lr=0.01, decay=0.0001, clipvalue=0.5)
    clf.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    return clf

In [ ]:
clf = KerasClassifier(build_fn=createNetowrk, epochs=100, batch_size=10)
clf

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
res = cross_val_score(estimator=clf, X=df, y=classes, cv=kfold, scoring='accuracy')

In [ ]:
res

In [ ]:
res.mean(), res.std()

## # 2.2 - Grid Search

In [ ]:
def createNetowrkGridSearch(units, dropout, optimizer, loss, activation):
    clf = Sequential()
    clf.add(Dense(input_dim=30, units=units, activation=activation,
                  kernel_initializer='random_uniform', name='dense_1'))
    clf.add(Dropout(rate=dropout, seed=0, name='dropout_1'))
    clf.add(Dense(units=units, activation=activation,
                  kernel_initializer='random_uniform', name='dense_2'))
    clf.add(Dropout(rate=dropout, seed=0, name='dropout_2'))
    clf.add(Dense(units=units, activation=activation,
                  kernel_initializer='random_uniform', name='dense_3'))
    clf.add(Dropout(rate=dropout, seed=0, name='dropout_3'))
    clf.add(Dense(units=units, activation=activation,
                  kernel_initializer='random_uniform', name='dense_4'))
    clf.add(Dense(units=1, activation='sigmoid', name='output'))
    clf.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    return clf

In [ ]:
clf_gridsearch = KerasClassifier(build_fn=createNetowrkGridSearch)

In [ ]:
params = {
    'epochs': [80, 100],
    'batch_size': [5, 10],
    'units': [30, 20],
    'dropout': [0.1, 0.20],
    'optimizer': ['adam', 'rmsprop'],
    'loss': ['binary_crossentropy', 'mean_squared_error'],
    'activation': ['tanh', 'relu']
}

In [ ]:
gdsearch = GridSearchCV(estimator=clf_gridsearch, param_grid=params,
                        scoring='accuracy', cv=5)

In [ ]:
fit = gdsearch.fit(X=df, y=classes) # This will take long

In [ ]:
best_params = fit.best_params_
best_acc = fit.best_score_

best_params, best_acc

## # 3.0 - Multiclass Classification

In [ ]:
df = pd.read_csv('datasets/iris.csv')

In [ ]:
data = df.iloc[:, 0:4].values

In [ ]:
labelEnc = LabelEncoder()

classes = labelEnc.fit_transform(df.iloc[:, 4].values)
classes_dummy = np_utils.to_categorical(classes)

In [ ]:
trainX, testX, trainY, testY = train_test_split(data, classes_dummy, test_size=0.25, random_state=0)

f'trainX: {trainX.shape}, trainY:{trainY.shape}, testX: {testX.shape}, testY: {testY.shape}'

In [ ]:
def seqNet():
    clf = Sequential()
    clf.add(Dense(units=4, activation='relu', input_dim=4))
    clf.add(Dense(units=8, activation='relu'))
    clf.add(Dense(units=8, activation='relu'))
    clf.add(Dense(units=8, activation='relu'))
    clf.add(Dense(units=3, activation='softmax'))
    clf.compile(optimizer='adam',
                loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return clf

In [ ]:
clf = seqNet()
clf

In [ ]:
clf.fit(x=trainX, y=trainY, epochs=1500, verbose=1)

In [ ]:
res = clf.evaluate(x=testX, y=testY)
res

In [ ]:
pred = (clf.predict(x=testX) > 0.5)

In [ ]:
_testY_argmax = [np.argmax(x) for x in testY] # returns the index

In [ ]:
_pred_argmax = [np.argmax(x) for x in pred]

In [ ]:
(
    confusion_matrix(y_true=_testY_argmax, y_pred=_pred_argmax),
    accuracy_score(y_true=_testY_argmax, y_pred=_pred_argmax)
)

## # 3.1 - Cross validation

In [ ]:
clf = KerasClassifier(build_fn=seqNet, epochs=1500)

In [ ]:
res = cross_val_score(estimator=clf, X=data, y=classes, cv=10, scoring='accuracy')

In [ ]:
res.mean(), res.std()

## # 4.0 - Regression

In [ ]:
df = pd.read_csv('datasets/autos.csv', encoding='iso-8859-1')

In [ ]:
df.drop(labels=['dateCrawled', 'dateCreated', 'nrOfPictures', 'postalCode',
                'name', 'lastSeen', 'seller', 'offerType'],
        axis=1, inplace=True)

In [ ]:
df.price.describe()

In [ ]:
df.price.mean()

In [ ]:
df[df.price == 0].shape

In [ ]:
df[df.price > 350000].shape

In [ ]:
i1 = df[df.price <= 10]
i1.shape

In [ ]:
i2 = df[df.price > 350000]
i2.shape

In [ ]:
df = df[df.price > 10]
df = df[df.price < 350000]

df.shape

In [ ]:
(
    df[pd.isnull(df['vehicleType'])].shape[0],
    df[pd.isnull(df['gearbox'])].shape[0],
    df[pd.isnull(df['model'])].shape[0],
    df[pd.isnull(df['fuelType'])].shape[0],
    df[pd.isnull(df['notRepairedDamage'])].shape[0]
)

In [ ]:
df['vehicleType'].value_counts()

In [ ]:
df['gearbox'].value_counts()

In [ ]:
df['model'].value_counts()

In [ ]:
df['fuelType'].value_counts()

In [ ]:
df['notRepairedDamage'].value_counts()

In [ ]:
fill = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzin',
    'notRepairedDamage': 'nein'
}

In [ ]:
df.fillna(value=fill, inplace=True)

In [ ]:
df.columns

In [ ]:
feat = df.iloc[:, 1:13].values
target = df.iloc[:, 0].values

f'features: {feat.shape}, target: {target.shape}'

In [ ]:
labelEnc = LabelEncoder()

In [ ]:
feat[:, 0] = labelEnc.fit_transform(feat[:, 0])
feat[:, 1] = labelEnc.fit_transform(feat[:, 1])
feat[:, 3] = labelEnc.fit_transform(feat[:, 3])
feat[:, 5] = labelEnc.fit_transform(feat[:, 5])
feat[:, 8] = labelEnc.fit_transform(feat[:, 8])
feat[:, 9] = labelEnc.fit_transform(feat[:, 9])
feat[:, 10] = labelEnc.fit_transform(feat[:, 10])

In [ ]:
feat.shape

In [ ]:
onehotEnc = OneHotEncoder(categorical_features=[0, 1, 3, 5, 8, 9, 10])

In [ ]:
feat = onehotEnc.fit_transform(feat).toarray()

In [ ]:
feat.shape

In [ ]:
reg = Sequential()

In [ ]:
reg.add(Dense(units=316, activation='relu', input_dim=316))
reg.add(Dense(units=158, activation='relu'))
reg.add(Dense(units=32, activation='relu'))
reg.add(Dense(units=1, activation='linear'))

In [ ]:
reg.compile(optimizer='adam', loss='mean_absolute_error', metrics='mean_absolute_error')

In [ ]:
reg.fit(x=feat, y=target, epochs=10, batch_size=10)

In [ ]:
pred = reg.predict(feat)

In [ ]:
t_mean = target.mean()
p_mean = pred.mean()

f'real mean: {t_mean}, pred mean: {p_mean}, abs distance: {abs(t_mean - p_mean)}'

## # 4.1 - Cross validation

In [ ]:
def createNet():
    reg = Sequential()
    reg.add(Dense(units=158, activation='relu', input_dim=316))
    reg.add(Dense(units=158, activation='relu'))
    reg.add(Dense(units=1, activation='linear'))
    reg.compile(optimizer='adam', loss='mean_absolute_error', metrics='mean_absolute_error')
    return reg

In [ ]:
reg = KerasRegressor(build_fn=createNet, epochs=10, batch_size=10)

In [ ]:
res = cross_val_score(estimator=reg, X=feat, y=target, cv=10, scoring='neg_mean_absolute_error')

## # 5 - Multi regression

In [ ]:
df = pd.read_csv('datasets/games.csv')
df.shape

In [ ]:
nans(df).shape

In [ ]:
df.head()

In [ ]:
df.drop(labels=['Name', 'Other_Sales', 'Global_Sales', 'Developer'], axis=1, inplace=True)

In [ ]:
df.dropna(axis=0, inplace=True)
df.shape

In [ ]:
df = df.loc[df['NA_Sales'] > 1]
df = df.loc[df['EU_Sales'] > 1]
df.shape

In [ ]:
df.columns

In [ ]:
features = df.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values
features.shape

In [ ]:
features[0]

In [ ]:
sells_na = df.iloc[:, 4].values
sells_eu = df.iloc[:, 5].values
sells_jp = df.iloc[:, 6].values

f'Sells: NA = {sells_na.shape[0]}, EU = {sells_eu.shape[0]}, JP = {sells_jp.shape[0]}'

In [ ]:
labelEncoder = LabelEncoder()

In [ ]:
features[:, 0] = labelEncoder.fit_transform(features[:, 0])
features[:, 2] = labelEncoder.fit_transform(features[:, 2])
features[:, 3] = labelEncoder.fit_transform(features[:, 3])
features[:, 8] = labelEncoder.fit_transform(features[:, 8])

In [ ]:
features[0]

In [ ]:
oneHot = OneHotEncoder(categorical_features=[0, 2, 3, 8])

In [ ]:
features = oneHot.fit_transform(features).toarray()
features.shape

In [ ]:
pd.DataFrame(features).head()

## # 5.1 - Network

In [ ]:
# input
input_layer = Input(shape=(61,), name='input')
# hidden
hidden_1 = Dense(units=32, activation='sigmoid', name='h1')(input_layer)
hidden_2 = Dense(units=32, activation='sigmoid', name='h2')(hidden_1)
# output
output_layer_1 = Dense(units=1, activation='linear', name='o1')(hidden_2)
output_layer_2 = Dense(units=1, activation='linear', name='o2')(hidden_2)
output_layer_3 = Dense(units=1, activation='linear', name='o3')(hidden_2)

In [ ]:
reg = Model(inputs=input_layer, outputs=[output_layer_1, output_layer_2, output_layer_3])
reg.compile(optimizer='adam', loss='mse')

In [ ]:
reg.summary()

In [ ]:
reg.fit(x=features, y=[sells_na, sells_eu, sells_jp], epochs=10000, batch_size=10)

In [ ]:
pred_na, pred_eu, pred_jp = reg.predict(features)

In [ ]:
pred_na.mean(), pred_eu.mean(), pred_jp.mean()

In [ ]:
f'Mean distances: \
NA = {abs(sells_na.mean() - pred_na.mean())}, \
EU = {abs(sells_eu.mean() - pred_eu.mean())}, \
JP = {abs(sells_jp.mean() - pred_jp.mean())}'

## # 6.0 - Convolutional Networks

In [ ]:
(trainx, trainy), (testx, testy) = mnist.load_data()

In [ ]:
trainx.shape, trainy.shape, testx.shape, testy.shape

In [ ]:
plt.title(label=f'Classe: {trainy[123]}')
plt.imshow(X=trainx[123])

In [ ]:
features_train = trainx.reshape(trainx.shape[0], 28, 28, 1)
features_train.shape

In [ ]:
features_test = testx.reshape(testx.shape[0], 28, 28, 1)
features_test.shape

In [ ]:
features_train = features_train.astype('float32')
features_test = features_test.astype('float32')

In [ ]:
features_train = features_train/255
features_test = features_test/255

In [ ]:
target_train = np_utils.to_categorical(y=trainy, num_classes=10)
target_test = np_utils.to_categorical(y=testy, num_classes=10)

target_train.shape, target_test.shape

In [ ]:
target_train[0]

## # 6.1 - Model

Output shape: $floor(\frac{n-f+2p}{s}+1) $


In [ ]:
# calc output shape
def outputShape(n, f, padding=0, stride=1):
    """
    This works for square matrices: n == height == width
    """
    return int(((n - f + 2 * padding ) / stride) + 1)

In [ ]:
def model(verbose=True):
    clf = Sequential()
    clf.add(layer=Conv2D(filters=32, kernel_size=(3, 3), activation='relu', name='conv',
                    input_shape=(28, 28, 1))) #outputs 26x26
    clf.add(layer=MaxPooling2D(pool_size=(2, 2), name='maxpool')) #outputs 13x13
    clf.add(layer=Flatten(name='flatten'))
    clf.add(layer=Dense(units=128, activation='relu', name='dense'))
    clf.add(layer=Dense(units=10, activation='softmax', name='softmax'))
    clf.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    if verbose:
        print(clf.summary())
    return clf

In [ ]:
clf = model()

In [ ]:
clf.fit(x=features_train, y=target_train, batch_size=100, epochs=10,
        validation_data=(features_test, target_test))

In [ ]:
clf.evaluate(features_test, target_test)

## # 6.2 - Model upgrade

In [ ]:
def model_upgraded(verbose=True):
    clf = Sequential()
    clf.add(layer=Conv2D(filters=32, kernel_size=(3, 3), activation='relu', name='conv1',
                    input_shape=(28, 28, 1)))
    clf.add(BatchNormalization(name='batchnorm1'))
    clf.add(layer=MaxPooling2D(pool_size=(2, 2), name='maxpool1'))
    clf.add(layer=Conv2D(filters=32, kernel_size=(3, 3), activation='relu', name='conv2'))
    clf.add(BatchNormalization(name='batchnorm2'))
    clf.add(layer=MaxPooling2D(pool_size=(2, 2), name='maxpool2'))
    clf.add(layer=Flatten(name='flatten'))
    clf.add(layer=Dense(units=128, activation='relu', name='dense1'))
    clf.add(Dropout(rate=0.2, name='dropout1'))
    clf.add(layer=Dense(units=64, activation='relu', name='dense2'))
    clf.add(Dropout(rate=0.2, name='dropout2'))
    clf.add(layer=Dense(units=10, activation='softmax', name='softmax'))
    clf.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    if verbose:
        print(clf.summary())
    return clf

In [ ]:
clf = model_upgraded()

In [ ]:
clf.fit(x=features_train, y=target_train, batch_size=100, epochs=10,
        validation_data=(features_test, target_test))

## # 6.3 - Cross validation

In [ ]:
seed = 42
np.random.seed(seed)

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [ ]:
results = []
for itrain, itest in kfold.split(X=features_train, y=np.zeros(shape=(target_train.shape[0], 1))):
    clf = model_upgraded(verbose=False)
    clf.fit(x=features_train[itrain], y=target_train[itrain], batch_size=100, epochs=5)
    prec = clf.evaluate(features_train[itest], target_train[itest])
    results.append(prec[1])

In [ ]:
np.array(results).mean()

## # 6.4 - Data Augmentation

In [ ]:
clf = model_upgraded(verbose=False)

In [ ]:
imggen_train = ImageDataGenerator(rotation_range=7, height_shift_range=0.07,
                            shear_range=0.2, zoom_range=0.2)

imggen_test = ImageDataGenerator()

In [ ]:
features_train_gen = imggen_train.flow(x=features_train, y=target_train, batch_size=128)

In [ ]:
features_test_gen = imggen_test.flow(x=features_test, y=target_test, batch_size=128)

In [ ]:
clf.fit_generator(generator=features_train_gen, steps_per_epoch=60000/128, epochs=5,
                 validation_data=features_test_gen, validation_steps=10000/128)

In [ ]:
clf.evaluate(features_test, target_test)

## # 6.5 - CIFAR-10

In [ ]:
(xtrain, ytrain), (xtest, ytest) = cifar10.load_data()

In [ ]:
plt.imshow(xtrain[666])
plt.title('Classe '+ str(ytrain[666]))

In [ ]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

In [ ]:
xtrain = xtrain.reshape(xtrain.shape[0], 32, 32, 3)
xtest = xtest.reshape(xtest.shape[0], 32, 32, 3)

In [ ]:
xtrain = xtrain.astype('float32')
xtest = xtest.astype('float32')

In [ ]:
xtrain /= 255
xtest /= 255

In [ ]:
ytrain = np_utils.to_categorical(ytrain, 10)
ytest = np_utils.to_categorical(ytest, 10)

In [ ]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

In [ ]:
def modelCifar10():
    clf = Sequential()
    clf.add(layer=Conv2D(filters=32, kernel_size=(3, 3), activation='relu', name='conv1',
                    input_shape=(32, 32, 3)))
    clf.add(BatchNormalization(name='batchnorm1'))
    clf.add(layer=MaxPooling2D(pool_size=(2, 2), name='maxpool1'))
    clf.add(layer=Conv2D(filters=32, kernel_size=(3, 3), activation='relu', name='conv2'))
    clf.add(BatchNormalization(name='batchnorm2'))
    clf.add(layer=MaxPooling2D(pool_size=(2, 2), name='maxpool2'))
    clf.add(layer=Flatten(name='flatten'))
    clf.add(layer=Dense(units=128, activation='relu', name='dense1'))
    clf.add(Dropout(rate=0.2, name='dropout1'))
    clf.add(layer=Dense(units=128, activation='relu', name='dense2'))
    clf.add(Dropout(rate=0.2, name='dropout2'))
    clf.add(layer=Dense(units=10, activation='softmax', name='softmax'))
    clf.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
#     print(clf.summary())
    return clf

In [ ]:
clf = modelCifar10()

In [ ]:
clf.fit(x=xtrain, y=ytrain, batch_size=32, epochs=5, validation_data=(xtest, ytest))

## # 7.0 - Cats & dogs

In [ ]:
train_dir = 'datasets/catsAndDogs/training_set/'
test_dir = 'datasets/catsAndDogs/test_set/'

In [ ]:
def catDogClf(verbose=False):
    clf = Sequential(name='catDogClf')

    clf.add(layer=Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name='conv1',
                    input_shape=(128, 128, 3)))
    clf.add(BatchNormalization(name='batchnorm1'))
    clf.add(layer=MaxPooling2D(pool_size=(2, 2), name='maxpool1'))

    clf.add(layer=Conv2D(filters=128, kernel_size=(3, 3), activation='relu', name='conv2'))
    clf.add(BatchNormalization(name='batchnorm2'))
    clf.add(layer=MaxPooling2D(pool_size=(2, 2), name='maxpool2'))
    
#     clf.add(layer=Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name='conv3'))
#     clf.add(BatchNormalization(name='batchnorm3'))
#     clf.add(layer=MaxPooling2D(pool_size=(2, 2), name='maxpool3'))
    
#     clf.add(layer=Conv2D(filters=128, kernel_size=(3, 3), activation='relu', name='conv4'))
#     clf.add(BatchNormalization(name='batchnorm4'))
#     clf.add(layer=MaxPooling2D(pool_size=(2, 2), name='maxpool4'))

    clf.add(layer=Flatten(name='flatten'))

    clf.add(layer=Dense(units=32, activation='relu', name='dense1'))
    clf.add(Dropout(rate=0.2, name='dropout1'))

#     clf.add(layer=Dense(units=64, activation='relu', name='dense2'))
#     clf.add(Dropout(rate=0.2, name='dropout2'))

    clf.add(layer=Dense(units=1, activation='sigmoid', name='sigmoid'))

    clf.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
    
    if verbose:
        print(clf.summary())
    
    return clf

In [ ]:
clf = catDogClf()

In [ ]:
imggen_train = ImageDataGenerator(rescale=1./255, rotation_range=7, horizontal_flip=True,
                                 shear_range=0.2, height_shift_range=0.07, zoom_range=0.2)
imggen_test = ImageDataGenerator(rescale=1./255)

In [ ]:
# It should contain one subdirectory per class.
train_data = imggen_train.flow_from_directory(directory=train_dir, target_size=(128, 128),
                                              batch_size=8, class_mode='binary')
test_data = imggen_test.flow_from_directory(directory=test_dir, target_size=(128, 128),
                                              batch_size=8, class_mode='binary')

In [ ]:
train_data.class_indices

In [ ]:
clf.fit_generator(generator=train_data, steps_per_epoch=4000/8, epochs=10,
                  validation_data=test_data, validation_steps=1000/8)

In [ ]:
test_images = "apolinho2.jpeg bruce1.jpeg not_bruce.jpeg \
                apolinho3.jpeg german.jpg".split()

for i in test_images:
    test_img_file = f'datasets/catsAndDogs/{i}'
    test_img = image.load_img(path=test_img_file, target_size=(128, 128))
    plt.imshow(test_img)

    test_img = image.img_to_array(img=test_img)
    test_img /= 255
    test_img = np.expand_dims(a=test_img, axis=0)
    test_img.shape

    pred = clf.predict(x=test_img)
    print(pred)

    if pred[0] < 0.5:
        print('Cat')
    else:
        print('Dog')
    
    plt.show()

## # 8.1 - Homer and Bart - Dense

In [ ]:
df = pd.read_csv(filepath_or_buffer='datasets/personagens.csv')
df.shape

In [ ]:
df.head()

In [ ]:
features = df.iloc[:, 0:6].values
features.shape

In [ ]:
target = df.iloc[:, 6].values
target.shape

In [ ]:
labelencoder = LabelEncoder()
target = labelencoder.fit_transform(target)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(features, target,
                                                test_size=0.25, random_state=0)

In [ ]:
(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)

In [ ]:
def simpsonsDense():
    clf = Sequential()
    clf.add(layer=Dense(units=32, input_dim=6, activation='relu'))
    clf.add(layer=Dense(units=64, activation='relu'))
    clf.add(layer=Dense(units=32, activation='relu'))
    clf.add(layer=Dense(units=1, activation='sigmoid'))
    clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return clf    

In [ ]:
clf = simpsonsDense()

In [ ]:
clf.fit(x=xtrain, y=ytrain, batch_size=10, epochs=500,
        validation_data=(xtest, ytest))

## # 8.1 - Homer and Bart - Conv

In [ ]:
train_gen = ImageDataGenerator(rescale = 1./255, rotation_range=7, 
                                         horizontal_flip = True, shear_range=0.2,
                                         height_shift_range=0.07, zoom_range=0.2)
test_gen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_data = train_gen.flow_from_directory(directory='datasets/dataset_personagens/training_set',
                                            target_size=(64, 64),
                                            batch_size=10,
                                            class_mode='binary')

In [ ]:
test_data = test_gen.flow_from_directory(directory='datasets/dataset_personagens/test_set',
                                            target_size=(64, 64),
                                            batch_size=10,
                                            class_mode='binary')

In [ ]:
def simpsonsCONV():
    clf = Sequential()
    
    clf.add(layer=Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
    clf.add(layer=MaxPooling2D(pool_size = (2, 2)))
    
    clf.add(layer=Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
    clf.add(layer=MaxPooling2D(pool_size = (2, 2)))
    
    clf.add(layer=Flatten())
    
    clf.add(layer=Dense(units=32, activation='relu'))
    clf.add(layer=Dense(units=64, activation='relu'))
    clf.add(layer=Dense(units=32, activation='relu'))
    clf.add(layer=Dense(units=1, activation='sigmoid'))
    clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return clf

In [ ]:
clf = simpsonsCONV()

In [ ]:
clf.fit_generator(generator=train_data, steps_per_epoch=196/10, epochs=500,
                         validation_data=test_data, validation_steps=73/10)

## # 9.0 - RNNs

## # 9.1 - Stock prices

In [ ]:
dftrain = pd.read_csv('datasets/petr4_treinamento.csv')
dftest = pd.read_csv('datasets/petr4_teste.csv')

dftrain.shape, dftest.shape

In [ ]:
dftrain.head()

In [ ]:
dftrain.dropna(inplace=True)
dftrain.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
dftrain_norm = scaler.fit_transform(dftrain.iloc[:, 1:2])
dftrain_norm.shape

We shall have the network use 90 days previous prices to predict the future price, therefore:

$f([x_0, x_1, ... x_{90}]) = x_{91} = y$

In [ ]:
prev = []
real_price = []

for i in range(90, 1242):
    prev.append(dftrain_norm[i-90:i, 0])
    real_price.append(dftrain_norm[i, 0])

In [ ]:
prev = np.array(prev)
real_price = np.array(real_price)

In [ ]:
prev.shape, real_price.shape

In [ ]:
prev[1][89] == real_price[0]

In [ ]:
prev = np.reshape(a=prev, newshape=(prev.shape[0], prev.shape[1], 1))

In [ ]:
prev.shape

In [ ]:
def model_LSTM():
    keras.backend.clear_session()

    clf = Sequential()

    clf.add(layer=LSTM(units=100, return_sequences=True, input_shape=(prev.shape[1], 1)))
    clf.add(layer=Dropout(rate=0.3))

    clf.add(layer=LSTM(units=50, return_sequences=True))
    clf.add(layer=Dropout(rate=0.3))

    clf.add(layer=LSTM(units=50))
    clf.add(layer=Dropout(rate=0.3))

    clf.add(layer=Dense(units=1, activation='linear'))

    clf.compile(optimizer='rmsprop',
                loss='mean_squared_error',
                metrics='mean_absolute_error')
    
    print(clf.summary())

    return clf

clf = model_LSTM()

In [ ]:
clf.fit(x=prev, y=real_price, batch_size=10, epochs=100)

## # 9.1.2 - Model evaluation

In [ ]:
dftest.head()

In [ ]:
test_data = dftest.iloc[:, 1:2].values
test_data.shape

In [ ]:
fulldf = pd.concat(objs=(dftrain['Open'], dftest['Open']), axis=0)
fulldf.shape

In [ ]:
fulldf.head()

In [ ]:
input_data = fulldf[len(fulldf) - len(dftest) - 90:].values.reshape(-1, 1)
input_data.shape

In [ ]:
input_data[0]

In [ ]:
input_data_norm = scaler.transform(input_data)
input_data_norm.shape

In [ ]:
input_data_norm[0]

In [ ]:
xtest = []
for i in range(90, 112):
    xtest.append(input_data_norm[i-90:i, 0])

In [ ]:
xtest = np.array(xtest)
xtest.shape

In [ ]:
xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
xtest.shape

In [ ]:
pred = clf.predict(xtest)
pred.mean()

In [ ]:
pred = scaler.inverse_transform(pred)
pred.mean()

In [ ]:
f'Abs mean error (real - pred) = {abs(test_data.mean() - pred.mean())}' # =< 0.18

## # 9.1.3 - Plotting

In [ ]:
plt.plot(test_data, color='red', label='real')
plt.plot(pred, color='blue', label='pred')
plt.title('stock prices')
plt.xlabel('time')
plt.ylabel('price (yahoo finance)')
plt.legend()
plt.show()

## # 9.2 - Stock prices (multiple input variables) and Callbacks

In [ ]:
dftrain = pd.read_csv('datasets/petr4_treinamento.csv')
dftest = pd.read_csv('datasets/petr4_teste.csv')

dftrain.shape, dftest.shape

In [ ]:
dftrain.dropna(inplace=True)
dftrain.shape

In [ ]:
train_data = dftrain.iloc[:, 1:7].values
train_data.shape

In [ ]:
mmaxScaler = MinMaxScaler(feature_range=(0,1))
train_data_scaled = mmaxScaler.fit_transform(train_data)

In [ ]:
train_features_scaled = []
train_target_scaled = []
for i in range(90, train_data.shape[0]):
    train_features_scaled.append(train_data_scaled[i-90:i, 0:6])
    train_target_scaled.append(train_data_scaled[i, 0])

len(train_features_scaled), len(train_target_scaled)

In [ ]:
train_features_scaled[1][89] == train_target_scaled[0]

In [ ]:
train_features_scaled = np.array(train_features_scaled)
train_target_scaled = np.array(train_target_scaled)

train_features_scaled.shape, train_features_scaled.shape

Each input is of shape (90, 6), i.e.: 90 days worth of 6 attributes, **for each** input register

In [ ]:
def clf_LSTM():

    keras.backend.clear_session()
    
    clf = Sequential()

    clf.add(layer=LSTM(units=100, return_sequences=True,
                       input_shape=(train_features_scaled.shape[1], train_features_scaled.shape[2])))

    clf.add(layer=Dropout(rate=0.25))

    clf.add(layer=LSTM(units=50, return_sequences=True))
    clf.add(layer=Dropout(rate=0.25))

    clf.add(layer=LSTM(units=100, return_sequences=True))
    clf.add(layer=Dropout(rate=0.25))
            
    clf.add(layer=(Dense(units=1, activation='linear')))

    clf.compile(optimizer='rmsprop', loss='mse', metrics='mae')

    return clf


clf = clf_LSTM()

In [ ]:
es = EarlyStopping(monitor='loss', min_delta=1e-10, patience=5, verbose=1)
rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, verbose=1)
mcp = ModelCheckpoint(filepath='sess9_2-weights.h5', save_best_only=True, monitor='loss',
                      verbose=1)

In [ ]:
clf.fit(x=train_features_scaled, y=train_target_scaled,
        epochs=10, batch_size=10, callbacks=[es, rlr, mcp],
        verbose=1)

## # 9.2.2 - Model evaluation

In [ ]:
fulldf = pd.concat(objs=(dftrain, dftest), axis=0)
fulldf.shape

In [ ]:
fulldf.drop(labels=['Date'], axis=1, inplace=True)
fulldf.shape

In [ ]:
test_features = fulldf[len(fulldf) - len(dftest) - 90:].values

In [ ]:
test_features_scaled = mmaxScaler.transform(test_features)
test_features_scaled.shape

In [ ]:
test_features = []
for i in range(90, test_features_scaled.shape[0]):
    test_features.append(test_features_scaled[i-90:i])

In [ ]:
test_features = np.array(test_features)
test_features.shape

In [ ]:
pred = clf.predict(test_features)
pred.shape

In [ ]:
# fit on trainig data, but choose only the 'Price' column
mmaxScaler_test = MinMaxScaler(feature_range=(0,1))
mmaxScaler_test.fit_transform(train_data[:, 0:1])

In [ ]:
pred = mmaxScaler_test.inverse_transform(pred[:, 0])
pred.shape

In [ ]:
pred[:5]

In [ ]:
test_target = dftest.iloc[:, 1:2].values
test_target.shape

In [ ]:
plt.plot(test_target, color='red', label='real')
plt.plot(pred, color='blue', label='pred')
plt.title('stock prices')
plt.xlabel('time')
plt.ylabel('price (yahoo finance)')
plt.legend()
plt.show()

## # 9.3 - Stock prices (multiple outputs)

In [2]:
dftrain = pd.read_csv('datasets/petr4_treinamento.csv')

dftrain.dropna(inplace=True)

dftrain.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,18.086271,30182600.0
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,18.738441,30552600.0
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,18.766001,36141000.0
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,18.444506,28069600.0
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,17.911745,29091300.0


In [3]:
train_features_open = dftrain.iloc[:, 1:2].values #['Open'].values
train_features_high = dftrain.iloc[:, 2:3].values #['High'].values

train_features_open.shape, train_features_high.shape

((1242, 1), (1242, 1))

In [4]:
# scaler_open = MinMaxScaler(feature_range=(0,1))
# train_features_open_scaled = scaler_open.fit_transform(train_features_open)
# train_features_open_scaled[:3]

In [5]:
# scaler_high = MinMaxScaler(feature_range=(0,1))
# train_features_high_scaled = scaler_high.fit_transform(train_features_high)
# train_features_high_scaled[:3]

Adjusting the range:

In [6]:
data_range = 90

In [7]:
train_input = np.array(
    [
        train_features_open[i-data_range:i, 0] 
        for i in range(data_range, train_features_open.shape[0])
    ]
)

train_output_open = train_features_open[data_range:, 0]
train_output_high = train_features_high[data_range:, 0]

train_input.shape, train_output_open.shape, train_output_high.shape

((1152, 90), (1152,), (1152,))

In [8]:
train_input[1][data_range-1] == train_output_open[0], train_output_open[0]

(True, 19.91)

In [9]:
train_input = train_input.reshape((train_input.shape[0], train_input.shape[1], 1))
train_input.shape

(1152, 90, 1)

In [10]:
train_output = np.column_stack((train_output_open, train_output_high))
train_output.shape

(1152, 2)

In [11]:
mcp = ModelCheckpoint(filepath='sess9_2-weights.h5', save_best_only=True, monitor='loss',
                      verbose=1)

In [33]:
def clf_LSTM():

    keras.backend.clear_session()
    
    clf = Sequential()
    
    clf.add(LSTM(units = 100, return_sequences = True,
                 input_shape=(train_input.shape[1], 1)))
    clf.add(Dropout(0.3))

    clf.add(LSTM(units = 50, return_sequences = True))
    clf.add(Dropout(0.3))

    clf.add(LSTM(units = 50, return_sequences = True))
    clf.add(Dropout(0.3))

    clf.add(LSTM(units = 50))
    clf.add(Dropout(0.3))

    clf.add(Dense(units = 2, activation = 'linear'))

    clf.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',
                      metrics = ['mean_absolute_error'])
    return clf


clf = clf_LSTM()

clf.fit(x=train_input, y=train_output, epochs=100, batch_size=10, callbacks=[mcp])

Epoch 1/100
116/116 [==============================] - ETA: 0s - loss: 69.6392 - mean_absolute_error: 7.1507
Epoch 00001: loss did not improve from 1.65228
116/116 [==============================] - 7s 57ms/step - loss: 69.6392 - mean_absolute_error: 7.1507
Epoch 2/100
116/116 [==============================] - ETA: 0s - loss: 18.9749 - mean_absolute_error: 3.4467
Epoch 00002: loss did not improve from 1.65228
116/116 [==============================] - 7s 57ms/step - loss: 18.9749 - mean_absolute_error: 3.4467
Epoch 3/100
116/116 [==============================] - ETA: 0s - loss: 7.3428 - mean_absolute_error: 2.0778
Epoch 00003: loss did not improve from 1.65228
116/116 [==============================] - 6s 55ms/step - loss: 7.3428 - mean_absolute_error: 2.0778
Epoch 4/100
116/116 [==============================] - ETA: 0s - loss: 4.8980 - mean_absolute_error: 1.7179
Epoch 00004: loss did not improve from 1.65228
116/116 [==============================] - 6s 56ms/step - loss: 4.8980 - 

Epoch 33/100
115/116 [============================>.] - ETA: 0s - loss: 2.1865 - mean_absolute_error: 1.1649
Epoch 00033: loss did not improve from 1.65228
116/116 [==============================] - 6s 54ms/step - loss: 2.1832 - mean_absolute_error: 1.1636
Epoch 34/100
116/116 [==============================] - ETA: 0s - loss: 2.5048 - mean_absolute_error: 1.2200
Epoch 00034: loss did not improve from 1.65228
116/116 [==============================] - 7s 56ms/step - loss: 2.5048 - mean_absolute_error: 1.2200
Epoch 35/100
115/116 [============================>.] - ETA: 0s - loss: 2.1377 - mean_absolute_error: 1.1482
Epoch 00035: loss did not improve from 1.65228
116/116 [==============================] - 6s 54ms/step - loss: 2.1447 - mean_absolute_error: 1.1502
Epoch 36/100
116/116 [==============================] - ETA: 0s - loss: 2.3500 - mean_absolute_error: 1.1770
Epoch 00036: loss did not improve from 1.65228
116/116 [==============================] - 7s 57ms/step - loss: 2.3500 - 

KeyboardInterrupt: 

In [13]:
dftest = pd.read_csv('datasets/petr4_teste.csv')

In [14]:
target_open = dftest.iloc[:, 1:2].values
target_high = dftest.iloc[:, 2:3].values

In [15]:
fulldf = pd.concat((dftrain['Open'], dftest['Open']), axis=0)
fulldf.shape

(1264,)

In [16]:
test_features = fulldf[len(fulldf) - len(dftest) - data_range:].values.reshape(-1, 1)
test_features.shape

(112, 1)

In [17]:
test_input = np.array(
    [
        test_features[i-data_range:i, 0] 
        for i in range(data_range, test_features.shape[0])
    ]
)

test_input = test_input.reshape((test_input.shape[0], test_input.shape[1], 1))
        
test_input.shape

(22, 90, 1)

## # 9.3.2 - Model evaluation

In [ ]:
pred = clf.predict(x=test_input)

In [ ]:
f'Distance open price: {abs(target_open.mean() - pred[:, 0].mean())}' # >= 0.41

In [ ]:
f'Distance high price: {abs(target_high.mean() - pred[:, 1].mean())}' # >= 0.40

In [ ]:
fig = plt.figure(figsize=(20,16))

plt.subplot(2, 2, 1)
plt.plot(target_open, color='red', label='actual open')
plt.plot(pred[:, 0], color='blue', label='pred open')
plt.title('stock prices')
plt.xlabel('time')
plt.ylabel('price (yahoo finance)')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(target_high, color='purple', label='actual high')
plt.plot(pred[:, 1], color='green', label='pred high')
plt.title('stock prices')
plt.xlabel('time')
plt.ylabel('price (yahoo finance)')
plt.legend()

plt.show()

## # 10.0 - Self Organizing Map